## 라이브러리

In [2]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 처리
plt.rc("font", family = "Malgun Gothic")

# 기호 처리
plt.rcParams["axes.unicode_minus"] = False

## 원본 데이터 오차 계산하기

원본 데이터의 계획과 실제 발주 수량 사이의 오차 계산(mae, mse, rsme)

In [3]:
# 빈 DataFrame 생성
all_df = pd.DataFrame(columns=['part_number', 'day', 'mae', 'mse', 'rmse'])

for num in [6, 15, 16, 29, 94]:
    # 데이터 불러오기
    data = pd.read_csv("./data_new/01_전처리후_파트별_데이터/part{}_data.csv".format(num))

    # 오차 리스트
    mae_list = []
    mse_list = []
    rmse_list = []

    # D+1~7 반복문
    for day in range(1, 8, 1):
        diff_list = []

        # D일계획과의 차이 계산
        for i in range(len(data) - day):
            diff = abs(data['D일 투입예정 수량(D일계획)'][i + day] - data['D+1일 투입예정 수량(Total)'][i])
            diff_list.append(diff)

        # mae 계산
        mae = sum(diff_list) / len(diff_list)
        mae_list.append(mae)
        # mse 계산
        mse = np.mean(np.square(diff_list))
        mse_list.append(mse)
        # rmse 계산
        rmse = np.sqrt(mse)
        rmse_list.append(rmse)

    # 파트별로 데이터프레임으로 만들기
    part_df = pd.DataFrame({
        'part_number': ['Part {}'.format(num)] * 7,
        'day': list(range(1, 8)),
        'mae': mae_list,
        'mse': mse_list,
        'rmse': rmse_list
    })

    # concat하기 전에 all_df가 비어있으면 part_df로 바꾸기
    if all_df.empty:
        all_df = part_df
    else:
        all_df = pd.concat([all_df, part_df], ignore_index=True)

all_df

,part_number,day,mae,mse,rmse
0,Part 6,1,50.125000,4293.333333,65.523533
1,Part 6,2,54.042553,4752.765957,68.940307
2,Part 6,3,57.456522,5252.413043,72.473533
3,Part 6,4,58.311111,5440.000000,73.756356
4,Part 6,5,59.886364,5696.068182,75.472301
5,Part 6,6,56.372093,5353.813953,73.169761
6,Part 6,7,52.166667,4907.595238,70.054231
7,Part 15,1,10.375000,212.250000,14.568802
8,Part 15,2,12.553191,275.957447,16.611967
9,Part 15,3,15.326087,360.369565,18.983402


In [10]:
# 저장하기
all_df.to_csv("./data_new/원본데이터_오차.csv", encoding="euc-kr", index=False)

---

## 최종 모델과 RMSE 평균 비교

In [4]:
# 원본 데이터 오차
origin_data = pd.read_csv("./data_new/원본데이터_오차.csv")

# 최종 모델 결과
model_before_data = pd.read_csv("./data_new/lstm_결과.csv")
model_data = pd.read_csv("./data_new/lstm_결과_튜닝후.csv")

In [9]:
# 모든 결과를 저장할 데이터프레임
final_rmse_df = pd.DataFrame()

# 원본 데이터 rmse 평균
result_origin = origin_data.groupby('part_number')['rmse'].mean().reset_index()

# 결과 데이터프레임에 열 방향으로 합치기
final_rmse_df = pd.concat([final_rmse_df, result_origin['rmse']], axis=1, ignore_index=True)

# model_before_data에서 'part_number'가 6, 15인 경우의 rmse 평균
filtered_before_data = model_before_data[model_before_data['part_number'].isin([6, 15])]
result_before = filtered_before_data.groupby('part_number')['rmse'].mean().reset_index()

# model_data에서 'part_number'가 16, 29, 94인 경우의 rmse 평균
filtered_model_data = model_data[model_data['part_number'].isin([16, 29, 94])]
result_model = filtered_model_data.groupby('part_number')['rmse'].mean().reset_index()

# 결과 데이터프레임에 행 방향으로 합치기
result_model_df = pd.concat([result_before, result_model], axis=0, ignore_index=True)

# 결과 데이터프레임에 열 방향으로 합치기
final_rmse_df = pd.concat([final_rmse_df, result_model_df['rmse']], axis=1, ignore_index=True)

# 컬럼명 설정
final_rmse_df.columns = ['origin', 'model']

# 저장하기
final_rmse_df.to_csv('./data_new/원본과_모델_rmse_비교.csv', index=False, encoding='utf-8-sig')

# 최종 결과 출력
final_rmse_df

,origin,model
0,17.022727,8.874576
1,28.706001,9.446987
2,188.820207,15.692715
3,71.341432,9.603837
4,23.762691,6.089528


In [10]:
# 원본과 최종 모델의 rmse 평균의 차이 계산
final_rmse_df['diff'] = final_rmse_df['origin'] - final_rmse_df['model']
final_rmse_df

,origin,model,diff
0,17.022727,8.874576,8.148151
1,28.706001,9.446987,19.259014
2,188.820207,15.692715,173.127491
3,71.341432,9.603837,61.737595
4,23.762691,6.089528,17.673163


## 결과 해석

- 부품 6과 15는 튜닝 전 모델을, 부품 16, 29, 96은 튜닝 후 모델을 사용하기로 결정함
- LSTM 모델을 적용한 결과, 원본 데이터와 비교하여 평균 RMSE가 감소함- 
개발한 예측 모델을 활용하면 발주 계획의 오차를 줄이고, 오차로 인한 발주 비용을 절감하여 더 경제적일 것으로 보임